# Run analysis without input files
In this example it is shown how Damages and Losses analyses can be run without input files, like resilience and damage curves.
This will make running large numbers of analyses for e.g. uncertainty much more convenient, because instead of preparing input files to the analyses objects can be passed to the analyses.
Related to https://github.com/Deltares/ra2ce/issues/654.

In [1]:
# Imports

from pathlib import Path
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.analysis.analysis_config_data.analysis_config_data import (
    AnalysisConfigData,
    AnalysisSectionDamages,
    AnalysisSectionLosses,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_damages_enum import (
    AnalysisDamagesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.analysis_losses_enum import (
    AnalysisLossesEnum,
)
from ra2ce.analysis.analysis_config_data.enums.damage_curve_enum import DamageCurveEnum
from ra2ce.analysis.analysis_config_data.enums.event_type_enum import EventTypeEnum
from ra2ce.analysis.analysis_config_data.enums.traffic_period_enum import (
    TrafficPeriodEnum,
)
from ra2ce.analysis.analysis_config_data.enums.trip_purpose_enum import TripPurposeEnum
from ra2ce.analysis.analysis_config_data.enums.weighing_enum import WeighingEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.network.network_config_data.network_config_data import (
    HazardSection,
    NetworkConfigData,
    NetworkSection,
)


c:\Users\klapwij\code\ra2ce\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = Path.cwd().parent.joinpath("data", "adaptation")

static_path = root_dir.joinpath("static")
hazard_path =static_path.joinpath("hazard")
network_path = static_path.joinpath("network")
output_path=root_dir.joinpath("output")

input_path = root_dir.joinpath("input")

## Configure the network

In [3]:
_network_section = NetworkSection(
    source= SourceEnum.SHAPEFILE,       
    primary_file = [network_path.joinpath("network.shp")], 
    file_id = "ID",
    link_type_column="highway",
    save_gpkg=True
)

_hazard = HazardSection(
    hazard_map=[file for file in hazard_path.glob("*.tif")],
    hazard_field_name= ["waterdepth"],
    aggregate_wl = AggregateWlEnum.MEAN,
    hazard_crs = "EPSG:4326",
)

_network_config_data = NetworkConfigData(
    root_path=root_dir,
    static_path=static_path,
    output_path=output_path,
    network=_network_section,
    hazard=_hazard
    )

## Configuring the analyses
First the analysis run needs to be configured in the standard way, except no paths to input files are provided.

In [4]:

_damages_section = AnalysisSectionDamages(
    analysis=AnalysisDamagesEnum.DAMAGES,
    event_type=EventTypeEnum.EVENT,
    damage_curve=DamageCurveEnum.MAN,
    save_gpkg=True,
    save_csv=True,
)

_multi_link_losses_section = AnalysisSectionLosses(
    analysis=AnalysisLossesEnum.MULTI_LINK_LOSSES,
    event_type=EventTypeEnum.EVENT,
    weighing=WeighingEnum.TIME,
    threshold=0,
    production_loss_per_capita_per_hour=42,
    hours_per_traffic_period=8,
    traffic_period=TrafficPeriodEnum.DAY,
    trip_purposes=[
        TripPurposeEnum.BUSINESS,
        TripPurposeEnum.COMMUTE,
        TripPurposeEnum.FREIGHT,
        TripPurposeEnum.OTHER,
    ],
    resilience_curves_file=None,
    traffic_intensities_file=None,
    values_of_time_file=None,
    save_gpkg=True,
    save_csv=True,
)

_analysis_config_data = AnalysisConfigData(
    root_path=root_dir,
    input_path=input_path,
    static_path=static_path,
    output_path=output_path,
    analyses=[
        _damages_section,
        _multi_link_losses_section,
    ],
    aggregate_wl=AggregateWlEnum.MEAN,
)

## Creating the handler

In [9]:
handler = Ra2ceHandler.from_config(_network_config_data, _analysis_config_data)

handler.configure()

2025-01-09 03:21:51 PM - [losses_input_data_reader_base.py:53] - root - WARNING - No csv file found at None. Using default values for <class 'ra2ce.analysis.losses.traffic_intensities.traffic_intensities.TrafficIntensities'>.
2025-01-09 03:21:51 PM - [losses_input_data_reader_base.py:53] - root - WARNING - No csv file found at None. Using default values for <class 'ra2ce.analysis.losses.traffic_intensities.traffic_intensities.TrafficIntensities'>.
2025-01-09 03:23:04 PM - [losses_input_data_reader_base.py:53] - root - WARNING - No csv file found at None. Using default values for <class 'ra2ce.analysis.losses.resilience_curves.resilience_curves.ResilienceCurves'>.
2025-01-09 03:23:04 PM - [losses_input_data_reader_base.py:53] - root - WARNING - No csv file found at None. Using default values for <class 'ra2ce.analysis.losses.resilience_curves.resilience_curves.ResilienceCurves'>.
2025-01-09 03:23:28 PM - [losses_input_data_reader_base.py:53] - root - WARNING - No csv file found at None.

## Defining the analysis inputs
In this example the data is read from files outside the analysis, but this might as well be replaced by creating/generating them by code.

In [6]:
from ra2ce.analysis.damages.damage_functions.manual_damage_functions_reader import ManualDamageFunctionsReader
from ra2ce.analysis.losses.resilience_curves.resilience_curves_reader import ResilienceCurvesReader
from ra2ce.analysis.losses.time_values.time_values_reader import TimeValuesReader
from ra2ce.analysis.losses.traffic_intensities.traffic_intensities_reader import TrafficIntensitiesReader

# Damages input data
_manual_damage_functions = ManualDamageFunctionsReader().read(input_path.joinpath("damage_functions"))

# Losses input data
_resilience_curves = ResilienceCurvesReader().read(input_path.joinpath("resilience_curve.csv"))
_values_of_time = TimeValuesReader().read(input_path.joinpath("values_of_time.csv"))
_traffic_intensities = TrafficIntensitiesReader(_analysis_config_data.network.file_id).read(input_path.joinpath("traffic_intensities.csv"))

## Adding the analysis inputs

In [7]:

from ra2ce.analysis.damages.damages import Damages
from ra2ce.analysis.losses.multi_link_losses import MultiLinkLosses


_damages = handler.analysis_collection.get_analysis(AnalysisDamagesEnum.DAMAGES)
assert isinstance(_damages, Damages)
_damages.manual_damage_functions = _manual_damage_functions

_losses = handler.analysis_collection.get_analysis(AnalysisLossesEnum.MULTI_LINK_LOSSES)
assert isinstance(_losses, MultiLinkLosses)
_losses.resilience_curves = _resilience_curves
_losses.values_of_time = _values_of_time
_losses.intensities = _traffic_intensities

## Running the analysis

In [8]:
handler.run_analysis()

2025-01-09 03:17:14 PM - [analysis_runner_factory.py:63] - root - WARNING - More than one runner available, computation time could be longer than expected.
2025-01-09 03:17:14 PM - [damage_network_base.py:121] - root - WARNING - Of the 235 road segments, only 26 had lane data, so for 209 the '
                                    lane data will be interpolated from the existing data
2025-01-09 03:17:14 PM - [damage_network_base.py:134] - root - WARNING - Interpolated the missing lane data as follows: {'secondary': 2.0, 'tertiary': 2.0, 'trunk': 2.0}
c:\Users\klapwij\code\ra2ce\.env\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


[DamagesResultWrapper(segment_based_result=AnalysisResult(analysis_result=     link_id   ID  infra_type  avgspeed  \
 0          1    1    tertiary      58.0   
 1         45   45    tertiary      60.0   
 2         12   12    tertiary      58.0   
 3        120  120    tertiary      58.0   
 4         11   11    tertiary      58.0   
 ..       ...  ...         ...       ...   
 230      253  253       trunk      50.0   
 231      246  246       trunk      50.0   
 232      249  249  trunk_link      50.0   
 233      248  248       trunk      50.0   
 234      257  257       trunk      50.0   
 
                                               geometry  lanes  length  \
 0    LINESTRING (34.87673 -19.85047, 34.87642 -19.8...    2.0    70.0   
 1    LINESTRING (34.88155 -19.84906, 34.88017 -19.8...    2.0   529.0   
 2    LINESTRING (34.87606 -19.85055, 34.87628 -19.8...    2.0    71.0   
 3    LINESTRING (34.87614 -19.85074, 34.87611 -19.8...    2.0    36.0   
 4    LINESTRING (34.87606 